In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from datetime import datetime
from dateutil.parser import parse
import copy

#시각화
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import *
from sklearn.metrics import accuracy_score, confusion_matrix, log_loss

##알고리즘 
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


In [67]:
# 다양한 오차 측정 지표를 확인하기 위한 함수 정의

from sklearn.metrics import *

def get_clf_eval(y_test, pred=None, pred_proba=None): 
    confusion = confusion_matrix(y_test, pred) 
    accuracy = accuracy_score(y_test, pred) 
    precision = precision_score(y_test, pred , average= 'weighted') 
    recall = recall_score(y_test, pred , average= 'weighted') 
    f1 = f1_score(y_test, pred , average= 'weighted') 
    
    print('정확도(accuracy): {0:.4f}, 정밀도(precision): {1:.4f}, 재현율(recall): {2:.4f}, f1_score: {3:.4f}'.format(accuracy, precision, recall, f1))
    
#     confusion = confusion_matrix(y_test, pred) 
#     accuracy = accuracy_score(y_test, pred) 
#     precision = precision_score(y_test, pred , average= 'macro') 
#     recall = recall_score(y_test, pred , average= 'macro') 
#     f1 = f1_score(y_test, pred , average= 'macro') 
    
#     print('정확도(accuracy): {0:.4f}, 정밀도(precision): {1:.4f}, 재현율(recall): {2:.4f}, f1_score: {3:.4f}'.format(accuracy, precision, recall, f1))
    
    return [accuracy, precision, recall, f1]

In [2]:
def Cut(Series , cuts):
    "구분할 iter가능한 변수와 구분 기준을 입력받으면 구분 기준 앞에서 부터 1로 구분해서 return 함"
    Cuts = copy.deepcopy(cuts)
    Cuts.append(np.inf)
    Cuts.insert(0,-1*np.inf)
    R = len(Cuts)
    label = range(1,R)
    return pd.cut(Series, Cuts, labels = label)

In [3]:
df = pd.read_csv("d:/공모전/IPO/IPO/CleanData/X_train_After_IPO_1.csv")

In [4]:
df['상장일'] = pd.to_datetime(df['상장일'])

In [5]:
df.set_index(["상장일"],inplace = True)

In [6]:
df.drop(['종목명','공모 시가총액'],axis = 1,inplace = True)

In [7]:
df['Cat'] = Cut(df['1_3_수익률'],[-0.2, -0.1, 0.1, 0.2])

In [8]:
train_list = []
test_list = []
train_start_date = '2009-04-01'
test_start_date = parse(str(train_start_date)).date() + relativedelta(years =3)
train_end_date = parse(str(test_start_date)).date() - relativedelta(days =1)
test_end_date = parse(str(train_end_date)).date() + relativedelta(years =1)


    
while True:
  
    train_list.append(df[train_start_date : train_end_date])
    test_list.append(df[test_start_date : test_end_date])
    
    train_start_date = parse(str(train_start_date)).date() + relativedelta(months=3)
    test_start_date = parse(str(test_start_date)).date() + relativedelta(months=3)
    train_end_date = parse(str(train_end_date)).date() + relativedelta(months=3)
    test_end_date = parse(str(test_end_date)).date() + relativedelta(months=3)
    
    if str(train_start_date) == '2018-04-01':
        break
    
    

In [46]:
per = test_list[0]['1_3_수익률']

In [47]:
y_testdata = pd.concat([per,y_test,pred_value] , axis = 1)

In [49]:
y_testdata.columns = ['실제 y 수익률','실제 y라벨링','예측 y 라벨링']

In [50]:
y_testdata

,실제 y 수익률,실제 y라벨링,예측 y 라벨링
상장일,,,
2012-05-04,-0.136201,2,1
2012-07-06,0.304659,5,3
2012-07-13,0.232143,5,3
2012-07-19,0.555683,5,3
2012-07-25,-0.005000,3,3
2012-07-26,-0.097928,3,3
2012-08-02,0.072398,3,3
2012-09-25,-0.347783,1,3
2012-09-27,0.641791,5,3


## 함수로 말고 그냥 바로 넣어줘서 반복문 돌림

In [90]:
y_data = []
score_list = []
feature_list = []

for i in range(0,len(train_list)):
    X_train = train_list[i].drop(['1_3_수익률','Cat'],axis =1)
    y_train = train_list[i]['Cat']
    X_test = test_list[i].drop(['1_3_수익률','Cat'],axis =1)
    y_test = test_list[i]['Cat']
    
        # 랜덤 포레스트 학습 및 별도의 테스트 셋으로 예측 성능 평가
    clf = RandomForestClassifier(random_state=0 , max_depth = 10 , min_samples_leaf = 4 ,\
                                min_samples_split =  4 , n_estimators = 400)
    
    clf.fit(X_train , y_train)
    train_pred = clf.predict(X_train)
    test_pred = clf.predict(X_test)
    
    # 성과 평가
    train_res = get_clf_eval(y_train, train_pred)
    test_res = get_clf_eval(y_test,test_pred)
    res = train_res + test_res

  
    #feature_importance
    feature_importance = clf.feature_importances_
    
    
    pred_value = pd.Series(test_pred,index = y_test.index)
    per = test_list[i]['1_3_수익률']
    y_testdata = pd.concat([per,y_test,pred_value] , axis = 1)
    y_testdata.columns = ['실제 y 수익률','실제 y라벨링','예측 y 라벨링']
    
    score_list.append(res)
    feature_list.append(feature_importance)
    y_data.append(y_testdata)
   

정확도(accuracy): 0.8774, 정밀도(precision): 0.8987, 재현율(recall): 0.8774, f1_score: 0.8735
정확도(accuracy): 0.5455, 정밀도(precision): 0.3732, 재현율(recall): 0.5455, f1_score: 0.4432
정확도(accuracy): 0.8248, 정밀도(precision): 0.8659, 재현율(recall): 0.8248, f1_score: 0.8082
정확도(accuracy): 0.5600, 정밀도(precision): 0.4533, 재현율(recall): 0.5600, f1_score: 0.5011
정확도(accuracy): 0.8162, 정밀도(precision): 0.8595, 재현율(recall): 0.8162, f1_score: 0.7772
정확도(accuracy): 0.6364, 정밀도(precision): 0.6364, 재현율(recall): 0.6364, f1_score: 0.6364
정확도(accuracy): 0.7970, 정밀도(precision): 0.8458, 재현율(recall): 0.7970, f1_score: 0.7716
정확도(accuracy): 0.4722, 정밀도(precision): 0.5694, 재현율(recall): 0.4722, f1_score: 0.3821
정확도(accuracy): 0.8110, 정밀도(precision): 0.8586, 재현율(recall): 0.8110, f1_score: 0.7927
정확도(accuracy): 0.4839, 정밀도(precision): 0.2341, 재현율(recall): 0.4839, f1_score: 0.3156
정확도(accuracy): 0.8167, 정밀도(precision): 0.8618, 재현율(recall): 0.8167, f1_score: 0.7990
정확도(accuracy): 0.4194, 정밀도(precision): 0.1759, 재현율(recall): 0.419

## 저장예시

In [137]:
df_score = pd.DataFrame(score_list,columns = ["정확도","정밀도","재현율","f1_score"]*2).T 

In [145]:
df_score.rename(columns = lambda x : "model_set_"+ str(x),inplace = True)

In [138]:
df_feature = pd.DataFrame(feature_list,columns = X_train.columns).T 

In [146]:
df_feature.rename(columns = lambda x : "model_set_"+ str(x),inplace = True)

### 수익률 , 실제 y 라벨링,  예측 y 라벨링은 리스트안에 36개 모델 담음

In [151]:
y_data[0]

,실제 y 수익률,실제 y라벨링,예측 y 라벨링
상장일,,,
2012-05-04,-0.136201,2,1
2012-07-06,0.304659,5,3
2012-07-13,0.232143,5,3
2012-07-19,0.555683,5,3
2012-07-25,-0.005000,3,3
2012-07-26,-0.097928,3,3
2012-08-02,0.072398,3,3
2012-09-25,-0.347783,1,3
2012-09-27,0.641791,5,3


### 평가지표에 대한 점수는 36개 모델을 열로 설정하고 하나의 데이터프레임으로 저장

In [147]:
df_score

,model_set_0,model_set_1,model_set_2,model_set_3,model_set_4,model_set_5,model_set_6,model_set_7,model_set_8,model_set_9,...,model_set_26,model_set_27,model_set_28,model_set_29,model_set_30,model_set_31,model_set_32,model_set_33,model_set_34,model_set_35
정확도,0.877419,0.824818,0.816176,0.796992,0.811024,0.816667,0.763636,0.863636,0.800000,0.768421,...,0.935135,0.902299,0.896552,0.914773,0.920635,0.909091,0.930108,0.898305,0.882022,0.868421
정밀도,0.898712,0.865904,0.859454,0.845761,0.858593,0.861849,0.829350,0.889676,0.847377,0.831027,...,0.939282,0.914201,0.911360,0.923849,0.930768,0.921299,0.937574,0.910288,0.899280,0.893063
재현율,0.877419,0.824818,0.816176,0.796992,0.811024,0.816667,0.763636,0.863636,0.800000,0.768421,...,0.935135,0.902299,0.896552,0.914773,0.920635,0.909091,0.930108,0.898305,0.882022,0.868421
f1_score,0.873456,0.808163,0.777230,0.771629,0.792713,0.798970,0.719235,0.850809,0.770762,0.724823,...,0.933500,0.900557,0.895954,0.913598,0.920435,0.908323,0.930076,0.896156,0.877344,0.858772
정확도,0.545455,0.560000,0.636364,0.472222,0.483871,0.419355,0.324324,0.319149,0.326087,0.368421,...,0.390625,0.328125,0.281250,0.262295,0.375000,0.344262,0.152778,0.308824,0.255814,0.368421
정밀도,0.373206,0.453333,0.636364,0.569444,0.234131,0.175858,0.105186,0.101856,0.106333,0.135734,...,0.284650,0.403589,0.169063,0.171311,0.311012,0.358286,0.489227,0.544340,0.213455,0.378788
재현율,0.545455,0.560000,0.636364,0.472222,0.483871,0.419355,0.324324,0.319149,0.326087,0.368421,...,0.390625,0.328125,0.281250,0.262295,0.375000,0.344262,0.152778,0.308824,0.255814,0.368421
f1_score,0.443182,0.501053,0.636364,0.382081,0.315568,0.247801,0.158853,0.154427,0.160371,0.198381,...,0.328224,0.273260,0.190818,0.200841,0.310617,0.331748,0.117987,0.300154,0.190119,0.347222


## feature importance 값도 위와 똑같이 하나의 데이터프레임으로 저장

In [148]:
df_feature

,model_set_0,model_set_1,model_set_2,model_set_3,model_set_4,model_set_5,model_set_6,model_set_7,model_set_8,model_set_9,...,model_set_26,model_set_27,model_set_28,model_set_29,model_set_30,model_set_31,model_set_32,model_set_33,model_set_34,model_set_35
매출총이익률,0.065722,0.057221,0.058201,0.056426,0.053276,0.051903,0.056848,0.050952,0.060488,0.050908,...,0.054362,0.051725,0.059809,0.053336,0.053621,0.047461,0.047277,0.050957,0.048858,0.051323
유동비율,0.075453,0.083389,0.078996,0.082043,0.082224,0.067572,0.066280,0.061350,0.058014,0.057970,...,0.059691,0.052774,0.059012,0.070591,0.066270,0.063902,0.063364,0.061727,0.058212,0.066480
부채비율,0.075703,0.070058,0.089679,0.085525,0.098664,0.085664,0.067595,0.066942,0.072980,0.072256,...,0.049903,0.055640,0.055538,0.060981,0.057732,0.058216,0.047122,0.049051,0.043835,0.049606
매출액증가율,0.058383,0.065604,0.063888,0.061987,0.061890,0.063165,0.067270,0.064127,0.059756,0.060817,...,0.050597,0.059292,0.054820,0.054265,0.066389,0.057359,0.052997,0.058150,0.058414,0.054701
1개월 후 PER,0.073110,0.069256,0.066770,0.063135,0.058843,0.059479,0.062310,0.056218,0.048060,0.054489,...,0.081637,0.073606,0.064527,0.066271,0.065853,0.055320,0.066495,0.068975,0.078334,0.064496
1개월 후 PBR,0.081528,0.081858,0.100442,0.105572,0.082336,0.080927,0.074245,0.082446,0.092097,0.079441,...,0.095385,0.084791,0.086670,0.094303,0.086080,0.078182,0.073519,0.064891,0.059743,0.064835
1개월 후 EV/EBITDA,0.057876,0.048980,0.053830,0.056875,0.059119,0.056875,0.044813,0.054863,0.061104,0.061967,...,0.080096,0.082169,0.073039,0.071102,0.068860,0.062292,0.065504,0.068528,0.080427,0.075911
1개월 평균 거래량회전율,0.064000,0.061949,0.065799,0.067375,0.057979,0.061376,0.060628,0.058170,0.046187,0.051773,...,0.078775,0.066016,0.069274,0.064294,0.065413,0.074570,0.069500,0.064672,0.061052,0.067670
1개월 평균 일중변동률,0.054030,0.053319,0.059967,0.053492,0.061234,0.063808,0.058497,0.057937,0.057109,0.067511,...,0.091066,0.094442,0.094571,0.087984,0.093014,0.097150,0.093434,0.090470,0.095851,0.089737
1개월 시가총액회전율,0.062830,0.052778,0.061987,0.062238,0.064886,0.053630,0.062408,0.054571,0.056228,0.056446,...,0.074387,0.067695,0.068331,0.062532,0.063339,0.066669,0.064210,0.060981,0.062437,0.059316
